# 견적서 HQ

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\Windows\Fonts\\malgun.ttf").get_name()
plt.rc("font", family=font_name)
import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

from datetime import datetime
import cv2
import zipfile
import shutil
import os
import glob
import json
from tqdm.notebook import tqdm

## 1) 견적서 json 가져오기

In [2]:
label_list = glob.glob("C:\\eun01\\car_repair\\training\\as_labeling_data\\*")
print(len(label_list))

125006


In [3]:
label_list[40].split("\\")[-1][:-5]

'as-0000076'

In [220]:
with open(label_list[6],'rt', encoding='UTF8') as file:
    contents=json.loads(file.read())

In [14]:
contents["차량정보"]["차량명칭"]

'BMW 2시리즈'

In [16]:
contents["차량정보"]["제작사/차종"]

'BMW'

In [140]:
category_id=[]
made=[]
model=[]

for file_name in tqdm(label_list):
    with open(file_name,'rt', encoding='UTF8') as file:
        contents=json.loads(file.read())
    category_id.append(file_name.split("\\")[-1][:-5])
    made.append(contents["차량정보"]["제작사/차종"])
    model.append(contents["차량정보"]["차량명칭"])


data = pd.DataFrame({"category_id" : category_id, "made" : made, "model" : model})

  0%|          | 0/125006 [00:00<?, ?it/s]

In [141]:
data.head()

,category_id,made,model
0,as-0000002,FORD,포드
1,as-0000004,BMW,BMW 5시리즈
2,as-0000005,CITROEN,씨트로엥
3,as-0000009,BMW,BMW 520D
4,as-0000011,BMW,NEW BMW 740Li


#### 외제차 제외

In [142]:
kor_index = pd.read_csv("kor_car.csv")
kor_index.rename(columns = {"file_name" : 'category_id'}, inplace = True)

In [143]:
data = pd.merge(data, kor_index, how='inner', on='category_id')

In [144]:
len(data)

121709

## 2) 차량 크기 추가

In [145]:
data["made"].unique()

array(['기아', '한국GM', '현대', '쌍용', '르노삼성', '현대 / RV', '현대 / 승용', '기아 / RV',
       '기아 / 승용', '쌍용 / RV', '한국GM / 승용', '르노삼성 / 승용', '한국GM / RV',
       '르노삼성 / RV', '기아 / 지프'], dtype=object)

In [118]:
car_repair1 = pd.read_csv("car_repair1.csv")
car_repair2 = pd.read_csv("car_repair2.csv")
car_repair3 = pd.read_csv("car_repair3.csv")

In [146]:
all_data = pd.concat([car_repair1, car_repair2, car_repair3], ignore_index=True)
all_data.sort_values("category_id")
all_data.head()

,image_id,category_id,segmentation,area,bbox,damage,year,color,repair,supercategory_name
0,2,as-0036229,"[[[[438, 504], [440, 443], [436, 419], [439, 4...",14977.0,"[336, 159, 198, 345]",Breakage,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
1,3,as-0036229,"[[[[469, 497], [448, 479], [458, 437], [461, 4...",7186.5,"[378, 209, 154, 288]",Breakage,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
2,4,as-0036229,"[[[[73, 596], [69, 588], [67, 582], [65, 575],...",749.5,"[53, 543, 31, 53]",Breakage,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
3,4,as-0036229,"[[[[107, 371], [108, 362], [445, 361], [795, 3...",13872.5,"[107, 356, 688, 29]",Scratched,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
4,5,as-0036229,"[[[[77, 390], [87, 375], [452, 351], [560, 356...",22816.0,"[77, 351, 690, 52]",Scratched,2020,Black,"['Rear bumper:coating,exchange']",Full-size car


In [147]:
# 외제차 분류 #
merge_car = pd.merge(all_data[["category_id","supercategory_name"]], kor_index, how='inner',on='category_id')
merge_car.head()

,category_id,supercategory_name
0,as-0036229,Full-size car
1,as-0036229,Full-size car
2,as-0036229,Full-size car
3,as-0036229,Full-size car
4,as-0036229,Full-size car


In [149]:
# 중복 없애주기 #
merge_car = merge_car.drop_duplicates(keep = 'first', ignore_index=True)
merge_car.sort_values("category_id", inplace=True, ignore_index=True)
merge_car.head()

,category_id,supercategory_name
0,as-0000025,Full-size car
1,as-0000027,Mid-size car
2,as-0000029,Mid-size car
3,as-0000034,Mid-size car
4,as-0000037,Mid-size car


### merge

In [150]:
merge_data = pd.merge(data, merge_car, how='inner',on='category_id')
merge_data.head()

,category_id,made,model,supercategory_name
0,as-0000025,기아,뉴카니발9인승(06),Full-size car
1,as-0000027,한국GM,말리부,Mid-size car
2,as-0000029,현대,아반떼AD(16),Mid-size car
3,as-0000034,기아,K5(18),Mid-size car
4,as-0000037,기아,올뉴쏘렌토,Mid-size car


In [152]:
merge_data = merge_data[["made", "model", "supercategory_name"]]
len(merge_data)

106949

In [153]:
# 중복 없애주기 #
merge_data = merge_data.drop_duplicates(keep = 'first', ignore_index=True)
merge_data.head()

,made,model,supercategory_name
0,기아,뉴카니발9인승(06),Full-size car
1,한국GM,말리부,Mid-size car
2,현대,아반떼AD(16),Mid-size car
3,기아,K5(18),Mid-size car
4,기아,올뉴쏘렌토,Mid-size car


In [154]:
merge_data.drop(merge_data[merge_data["model"]==""].index, axis=0, inplace=True)

In [155]:
merge_data["made"].unique()

array(['기아', '한국GM', '현대', '쌍용', '르노삼성', '현대 / RV', '현대 / 승용', '기아 / RV',
       '기아 / 승용', '쌍용 / RV', '한국GM / 승용', '르노삼성 / 승용', '한국GM / RV',
       '르노삼성 / RV', '기아 / 지프'], dtype=object)

In [156]:
name = ['한국GM', '한국GM / RV', '한국GM / 승용']

for n in name:
    idx = merge_data[merge_data["made"] == n].index
    merge_data["made"][idx] = "쉐보레"

In [157]:
merge_data.sort_values("made", inplace=True, ignore_index=True)
merge_data.head()

,made,model,supercategory_name
0,기아,뉴카니발9인승(06),Full-size car
1,기아,스토닉,Mid-size car
2,기아,프라이드5DR(15),Compact car
3,기아,K9(18),Full-size car
4,기아,뉴쏘렌토R(13),Mid-size car


In [158]:
# 회사명만 저장 #
for i in range(len(merge_data)):
    merge_data["made"][i] = merge_data["made"][i].split("/")[0].strip()

In [159]:
# 중복 없애주기 #
merge_data = merge_data.drop_duplicates(keep = 'first', ignore_index=True)
merge_data.head()

,made,model,supercategory_name
0,기아,뉴카니발9인승(06),Full-size car
1,기아,스토닉,Mid-size car
2,기아,프라이드5DR(15),Compact car
3,기아,K9(18),Full-size car
4,기아,뉴쏘렌토R(13),Mid-size car


In [160]:
merge_data["made"].unique()

array(['기아', '르노삼성', '쉐보레', '쌍용', '현대'], dtype=object)

In [163]:
merge_data.to_csv("car_model.csv", index=False, encoding="euc-kr")

## 변경한 차량 크기 합치기

In [166]:
car_final = pd.read_excel("car_model_final.xlsx")
car_final.head()

,made,model,supercategory_name,Unnamed: 3,연식
0,기아,뉴카니발9인승(06),VAN,NaN,2006.0
1,기아,스팅어(17),Mid-size car,NaN,2017.0
2,기아,스토닉,Mid-size car,NaN,NaN
3,기아,프라이드5DR(15),City car,NaN,2015.0
4,기아,K9(18),Full-size car,NaN,2018.0


In [168]:
car_final.drop("Unnamed: 3", axis=1, inplace=True)

In [221]:
car_final.head()

,made,model,supercategory_name,연식
0,기아,뉴카니발9인승(06),VAN,2006.0
1,기아,스팅어(17),Mid-size car,2017.0
2,기아,스토닉,Mid-size car,NaN
3,기아,프라이드5DR(15),City car,2015.0
4,기아,K9(18),Full-size car,2018.0


In [169]:
car_final["supercategory_name"].unique()

array(['VAN', 'Mid-size car', 'City car', 'Full-size car', 'SUV',
       'Compact car'], dtype=object)

In [170]:
# HQ 가져오기 #
repair1 = pd.read_csv("HQ_coating.csv")
repair2 = pd.read_csv("HQ_exchange.csv")
repair3 = pd.read_csv("HQ_sheet_metal.csv")

In [171]:
all_data = pd.concat([repair1, repair2, repair3], ignore_index=True)
all_data.sort_values("category_id")
all_data.head()

,category_id,part,damage,supercategory_name,repair,HQ
0,as-0000025,Bumper,Crushed,Full-size car,coating,1.97
1,as-0000025,Fender,Scratched,Full-size car,coating,3.26
2,as-0000025,Fender,Crushed,Full-size car,coating,3.26
3,as-0000025,Door,Scratched,Full-size car,coating,4.56
4,as-0000025,Door,Separated,Full-size car,coating,4.56


In [173]:
data.head()

,category_id,made,model
0,as-0000025,기아,뉴카니발9인승(06)
1,as-0000027,한국GM,말리부
2,as-0000029,현대,아반떼AD(16)
3,as-0000034,기아,K5(18)
4,as-0000037,기아,올뉴쏘렌토


In [175]:
merge_data = pd.merge(data[["category_id", "model"]], car_final, how='inner',on='model')
merge_data.head()

,category_id,model,made,supercategory_name,연식
0,as-0000025,뉴카니발9인승(06),기아,VAN,2006.0
1,as-0005803,뉴카니발9인승(06),기아,VAN,2006.0
2,as-0010405,뉴카니발9인승(06),기아,VAN,2006.0
3,as-0011018,뉴카니발9인승(06),기아,VAN,2006.0
4,as-0012163,뉴카니발9인승(06),기아,VAN,2006.0


In [178]:
merge_data.sort_values("category_id", inplace=True, ignore_index=True)
merge_data.head()

,category_id,model,made,supercategory_name,연식
0,as-0000025,뉴카니발9인승(06),기아,VAN,2006.0
1,as-0000027,말리부,쉐보레,Mid-size car,NaN
2,as-0000027,말리부,쉐보레,Mid-size car,NaN
3,as-0000029,아반떼AD(16),현대,City car,NaN
4,as-0000029,아반떼AD(16),현대,City car,NaN


In [180]:
all_data.drop("supercategory_name", axis=1, inplace=True)

In [183]:
merge_data2 = pd.merge(all_data, merge_data[["category_id", "supercategory_name"]], how='inner',on='category_id')
merge_data2.head()

,category_id,part,damage,repair,HQ,supercategory_name
0,as-0000025,Bumper,Crushed,coating,1.97,VAN
1,as-0000025,Fender,Scratched,coating,3.26,VAN
2,as-0000025,Fender,Crushed,coating,3.26,VAN
3,as-0000025,Door,Scratched,coating,4.56,VAN
4,as-0000025,Door,Separated,coating,4.56,VAN


In [188]:
merge = merge_data2[["category_id", "part", "damage", "repair", "supercategory_name", "HQ"]]
merge.head()

,category_id,part,damage,repair,supercategory_name,HQ
0,as-0000025,Bumper,Crushed,coating,VAN,1.97
1,as-0000025,Fender,Scratched,coating,VAN,3.26
2,as-0000025,Fender,Crushed,coating,VAN,3.26
3,as-0000025,Door,Scratched,coating,VAN,4.56
4,as-0000025,Door,Separated,coating,VAN,4.56


In [191]:
# 중복 없애주기 #
merge = merge.drop_duplicates(keep = 'first', ignore_index=True)
merge.head()

,category_id,part,damage,repair,supercategory_name,HQ
0,as-0000025,Bumper,Crushed,coating,VAN,1.97
1,as-0000025,Fender,Scratched,coating,VAN,3.26
2,as-0000025,Fender,Crushed,coating,VAN,3.26
3,as-0000025,Door,Scratched,coating,VAN,4.56
4,as-0000025,Door,Separated,coating,VAN,4.56


In [193]:
exchange = merge[merge["repair"]=="exchange"]
exchange.sort_values("category_id", inplace=True, ignore_index=True)

coating = merge[merge["repair"]=="coating"]
coating.sort_values("category_id", inplace=True, ignore_index=True)

sheet_metal = merge[merge["repair"]=="sheet_metal"]
sheet_metal.sort_values("category_id", inplace=True, ignore_index=True)

print(len(exchange))
print(len(coating))
print(len(sheet_metal))

579612
507070
254823


In [201]:
exchange.to_csv("HQ_exchange2.csv", index=False)
coating.to_csv("HQ_coating2.csv", index=False)
sheet_metal.to_csv("HQ_sheet_metal2.csv", index=False)